# Documention Agent

This notebook demonstrates a documention agent:
1. Video-to-protocol conversion using Vertex AI
2. With knowledge from documents and pictures that are loaded into cache

In [1]:
# %pip install google-cloud-storage
# %pip install --upgrade --user --quiet google-cloud-aiplatform

In [2]:
# %load_ext autoreload
%reload_ext autoreload
%autoreload 2

import configparser
import os
import sys
from pathlib import Path

from IPython.display import Markdown

path_to_append = Path(Path.cwd()).parent / "proteomics_specialist"
sys.path.append(str(path_to_append))
import video_to_protocol

config = configparser.ConfigParser()
config.read("../secrets.ini")

['../secrets.ini']

In [3]:
import configparser

import vertexai

config = configparser.ConfigParser()
config.read("../secrets.ini")

PROJECT_ID = config["DEFAULT"]["PROJECT_ID"]
vertexai.init(project=PROJECT_ID, location="europe-west9")  # europe-west9 is Paris

In [4]:
from google.cloud import storage

os.environ["GOOGLE_CLOUD_PROJECT"] = config["DEFAULT"]["PROJECT_ID"]

# Initialize Cloud Storage client
storage_client = storage.Client()
bucket_name = "mannlab_videos"
bucket = storage_client.bucket(bucket_name)

In [5]:
import datetime

from vertexai.generative_models import Part
from vertexai.preview import caching
from vertexai.preview.generative_models import GenerativeModel

MODEL_ID = "gemini-1.5-pro-001"

# Following: https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/context-caching/intro_context_caching_vertex_ai_sdk.ipynb

In [6]:
# Upload knowledge files to Google Cloud Storage
folder_path = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/knowledge_base"
subfolder_in_bucket = "knowledge"

knowledge_uris = []
for filename in os.listdir(folder_path):
    if filename.lower().endswith(
        (".jpg", ".jpeg", ".gif", ".bmp", ".tiff", ".tif", ".pdf")
    ):
        path = Path(folder_path) / filename
        try:
            file_uri = video_to_protocol.upload_video_to_gcs(
                path, bucket, subfolder_in_bucket
            )
            knowledge_uris.append(file_uri)
        except OSError as e:
            print(f"Error processing {filename}: {e}")

In [8]:
# Create cache with Vertex AI
import os
from collections import defaultdict

# Define supported file types with corresponding MIME types
MIME_TYPES = {
    ".pdf": "application/pdf",
    ".jpg": "image/jpeg",
    ".jpeg": "image/jpeg",
    ".png": "image/png",
}


def create_cached_content(
    knowledge_uris: list[str],
    model_id: str,
) -> list[Part]:
    """Create cached content from knowledge URIs.

    Args:
        knowledge_uris: list of URIs pointing to knowledge files
        bucket_name: Name of the GCS bucket
        subfolder_in_bucket: Subfolder path in the bucket
        model_id: ID of the model to use

    Returns:
        list of Part objects created from the knowledge URIs

    """
    contents = []
    file_counts = defaultdict(int)

    for file_path in knowledge_uris:
        path_obj = Path(file_path)
        file_ext = path_obj.suffix.lower()

        if file_ext in MIME_TYPES:
            mime_type = MIME_TYPES[file_ext]

            try:
                contents.append(Part.from_uri(file_path, mime_type=mime_type))
                file_counts[file_ext] += 1
            except (OSError, ValueError) as e:
                print(f"Error creating Part from {file_path}: {e}")
        else:
            print(f"Unsupported file extension: {file_ext}")

    print(f"Total files processed: {len(contents)}")
    for ext, count in file_counts.items():
        print(f"  {ext[1:].upper()}: {count}")

    if contents:
        cached_content = caching.CachedContent.create(
            model_name=model_id,
            contents=contents,
            ttl=datetime.timedelta(minutes=60),
        )
        print("Cached content created successfully!")
        return cached_content
    print("No matching files found. Cached content not created.")
    return None


cached_content = create_cached_content(knowledge_uris, model_id=MODEL_ID)

Total files processed: 40
  PDF: 40
Cached content created successfully!


In [9]:
# cached_content.delete()

In [10]:
print(cached_content.name)
# print(cached_content.resource_name)
# print(cached_content.model_name)
print(cached_content.create_time)
print(cached_content.expire_time)

4513486435927457792
2025-03-24 12:08:53.496338+00:00
2025-03-24 13:08:53.490116+00:00


In [11]:
model = GenerativeModel.from_cached_content(cached_content=cached_content)

In [12]:
video_path = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/ready_examples/Disconnect_IonOpticks_column_from_sample_line.mp4"
video_uri_input = video_to_protocol.upload_video_to_gcs(video_path, bucket)

In [13]:
prompt = """
You are a highly observant research assistant in Professor Matthias Mann's proteomics and mass spectrometry laboratory. Your expertise lies in detailed documentation of experimental procedures.

Analyze the video and reconstruct a step-by-step protocol by focusing on the actions in the video. Focus on user interactions with equipment, devices, and software. The goal is a clear, concise, unambiguous protocol reproducible by someone with no prior knowledge. "Think aloud" as if you were the researcher in the video that describes their work. Describe what you see at every secound. Take deep breath and think step-by-step. Answer direct.

For each action, describe:

* **Timestamp:** [timestamp]
* **Action:** [Specific Action/Change Observed (Include details of tools used, and observable results. (e.g., opening a lid, pressing a button, turning a knob, screwing/unscrewing, connecting/disconnecting, etc.))]

**Example:**
[02:15] timsControl Software: Mode changed from "Operate" to "Standby" by clicking the power button.
[03:45] Ion Source: NanoViper connector disconnected by unscrewing the nut counterclockwise.

"""

In [14]:
inputs = [
    prompt,
    "Input Video:",
    Part.from_uri(video_uri_input, mime_type="video/mp4"),
    "Observations:",
]

response = model.generate_content(inputs, generation_config={"temperature": 0})
observation = response.text
print(response.usage_metadata)
Markdown(observation)

prompt_token_count: 275833
candidates_token_count: 242
total_token_count: 276075
cached_content_token_count: 261613
prompt_tokens_details {
  modality: TEXT
  token_count: 256
}
prompt_tokens_details {
  modality: VIDEO
  token_count: 13965
}
prompt_tokens_details {
  modality: DOCUMENT
  token_count: 261612
}
candidates_tokens_details {
  modality: TEXT
  token_count: 242
}



The video shows a researcher connecting a separation column to a mass spectrometer. Here is a step-by-step protocol of the actions:

* **[0:14]** **Action:** The researcher reaches for the separation column and the emitter, which are already connected via a union.
* **[0:15]** **Action:** The researcher holds the separation column with their left hand and the emitter with their right hand.
* **[0:19]** **Action:** The researcher carefully inserts the emitter into the source.
* **[0:21]** **Action:** The researcher uses their right hand to tighten the propeller-shaped metal ring that secures the emitter to the source.
* **[0:33]** **Action:** The researcher uses pliers to tighten the propeller-shaped metal ring further.
* **[0:40]** **Action:** The researcher places the pliers in a yellow container.
* **[0:41]** **Action:** The researcher reaches for the transfer line.
* **[0:42]** **Action:** The researcher connects the transfer line to the separation column using a NanoViper connector. 
